### Imports

In [22]:
import pandas as pd 
import os
%matplotlib inline
   
from os import listdir
from utils import *
from tensorflow.keras.layers import Conv2D,Input,ZeroPadding2D,BatchNormalization,Flatten,Activation,Dense,MaxPooling2D
from tensorflow.keras.models import Sequential

### Loading Images

In [2]:
image_dir="/home/pedrod33/Desktop/MRSI/ano1s2/AA/Brain_Tumor_Detection/archive/brain_tumor_dataset/"

### Making directory for augmented images

**A directory is formed using os.makedirs() function for augmented images(yes/ no). Note- custom directory is obtained in outputs folder.**

In [3]:

#os.makedirs('../output/kaggle/working/augmented-images')
#os.makedirs('../output/kaggle/working/augmented-images/yes')
#os.makedirs('../output/kaggle/working/augmented-images/no')

## Augmentation of images 
**About the data:
The dataset contains 2 folders: yes and no which contains 253 Brain MRI Images. The folder yes contains 155 Brain MRI Images that are tumorous andno contains 98 Brain MRI Images that are non-tumorous.**

In [9]:
IMG_SIZE = 164
X, y = read_images()
X = crop_images(X)
X = resize_and_rescale(X, 164)
X_train, y_train, X_test, y_test = split_data(X, y)
X_train, y_train = augment_data(X_train,y_train, 164)
X_train, y_train, X_val, y_val = split_data(X_train, y_train,0.25)

## Visualization of data

In [11]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of validation examples = " + str(X_val.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))

number of training examples = 303
number of validation examples = 101
number of test examples = 51


In [37]:
def build_model(input_shape):
    model = Sequential()
    model.add(Input(input_shape)) 
    model.add(ZeroPadding2D((2, 2))) 
    #Conv2D(32, (7, 7), strides = (1, 1))(X_input)
    model.add(Conv2D(filters=32, kernel_size=4, input_shape=input_shape, strides = (1, 1)))
    model.add(BatchNormalization(axis = 3, name = 'bn0'))
    model.add(Activation('relu')) 
    
    model.add(MaxPooling2D((4, 4))) 
    model.add(MaxPooling2D((4, 4))) 
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid')) 
    
    return model

In [39]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE,3)
model=build_model(IMG_SHAPE)
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_3 (ZeroPaddi  (None, 168, 168, 3)      0         
 ng2D)                                                           
                                                                 
 conv2d_9 (Conv2D)           (None, 165, 165, 32)      1568      
                                                                 
 bn0 (BatchNormalization)    (None, 165, 165, 32)      128       
                                                                 
 activation (Activation)     (None, 165, 165, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 41, 41, 32)       0         
 )                                                               
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 32)     

In [40]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x=X_train, y=y_train, batch_size=32, epochs=22, validation_data=(X_val, y_val))

Epoch 1/22


ValueError: in user code:

    File "/home/pedrod33/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/pedrod33/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/pedrod33/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/pedrod33/.local/lib/python3.8/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/home/pedrod33/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/pedrod33/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 214, in assert_input_compatibility
        ra

In [ ]:
history = model.history.history

### Plotting of aaccuracy

In [ ]:
plot_metrics(history)